In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
n_collect = 200

In [3]:
def collect_vul(pars_post, n_collect=20, coverages=np.linspace(0, 0.5, 21), tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={tp: {'Coverage': k}}, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum()
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats

In [4]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)

    mss0, mss_stats0 = collect_vul(pars_post, n_collect=n_collect, tp='VulACF')
    mss1, mss_stats1 = collect_vul(pars_post, n_collect=n_collect, tp='PlainACF')
    mss = pd.concat([mss0, mss1])
    mss_stats = pd.concat([mss_stats0, mss_stats1])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_lo
dy_hi


In [3]:
def collect_fu(pars_post, n_collect=20, coverage=0.05, tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    intvs = {
        'Baseline': {},
        'Vul_0_0': {'VulACF': {'Coverage': coverage, 'FollowUp': 0, 'Duration': 0}},
        'Vul_2_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 2}},
        'Vul_2_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 2}},
        'Vul_3_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 3}},
        'Vul_3_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 3}},
    }
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
        
        for key, intv in intvs.items():
            _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum()
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [4]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fu(pars_post, n_collect=n_collect, coverage=0.03)
    mss.to_csv(f'{out_path}/Sim_VulACF_followup.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_followup_stats.csv')

dy_lo
dy_hi


In [5]:
mss_stats

,Key,Scenario,Pop0,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,ACF_Vul_Yield,ACF_Vul_TP,...,ACF_Vul_DR_Sl,ACF_Plain_Footfall,ACF_Plain_Screened,ACF_Plain_Confirmed,ACF_Plain_Yield,ACF_Plain_TP,ACF_Plain_DS_Fl,ACF_Plain_DR_Fl,ACF_Plain_DR_Sl,Coverage
0,6,Baseline,2.383838,0.194844,0.025620,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1,6,Vul_0_0,2.383838,0.186732,0.023809,7.436889,1.322738,0.344603,0.031613,0.009423,...,0.000463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
2,6,Vul_2_3,2.383838,0.177500,0.021606,9.385118,1.322738,0.344483,0.031466,0.009273,...,0.000459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
3,6,Vul_2_6,2.383838,0.179764,0.022135,9.381602,1.322738,0.344652,0.031674,0.009486,...,0.000470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
4,6,Vul_3_3,2.383838,0.175379,0.021058,10.309240,1.322738,0.344462,0.031439,0.009246,...,0.000461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1491,Vul_0_0,2.383309,0.174097,0.024086,7.438368,1.322444,0.333432,0.035519,0.011962,...,0.000550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1196,1491,Vul_2_3,2.383309,0.164458,0.021440,9.422998,1.322444,0.333308,0.035369,0.011809,...,0.000554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1197,1491,Vul_2_6,2.383309,0.166458,0.022012,9.424919,1.322444,0.333522,0.035628,0.012073,...,0.000565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1198,1491,Vul_3_3,2.383309,0.162110,0.020750,10.383307,1.322444,0.333285,0.035341,0.011780,...,0.000558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
